# MLflow Evaluation Demo: Multi-Turn Conversation with Session-Level Judges

This notebook demonstrates MLflow 3.7's evaluation capabilities by showing how to:
1. Set up session-level judges directly in the notebook
2. Use `mlflow.genai.evaluate()` API to evaluate conversations
3. Extract and interpret evaluation results

**Key Difference from v1**: This notebook showcases MLflow's evaluation methods (`make_judge`, `search_traces`, `genai.evaluate`) by keeping evaluation logic in the notebook rather than hidden in the agent class.

## What You'll Learn

- How to create session-level judges with `{{ conversation }}` template
- How to call `mlflow.genai.evaluate()` directly
- How to extract results from evaluation DataFrames
- Best practices for multi-turn conversation evaluation

---

## Setup: Import Dependencies

In [1]:
from genai.common.config import AgentConfig
from genai.agents.multi_turn.customer_support_agent_simple import CustomerSupportAgentSimple
from genai.agents.multi_turn.scenarios import get_scenario_printer_troubleshooting
from genai.agents.multi_turn.prompts import (
    get_coherence_judge_instructions,
    get_context_retention_judge_instructions,
    map_retention_to_score
)
from genai.common.mlflow_config import setup_mlflow_tracking
import mlflow
from mlflow.genai.judges import make_judge
from typing_extensions import Literal
import os
from pathlib import Path

# Load environment variables
try:
    from dotenv import load_dotenv
    env_file = Path(".env")
    if env_file.exists():
        load_dotenv(env_file)
        print(f"✓ Loaded environment variables from {env_file.absolute()}")
    else:
        print(f"ℹ️  No .env file found. Set environment variables manually.")
except ImportError:
    print("ℹ️  python-dotenv not installed. Set environment variables manually.")

## Convenience for display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

✓ Loaded environment variables from /Users/danny.chiao/personal/mlflow-misc/genai/agents/multi_turn/.env


## Configuration

In [2]:
# Provider and model configuration
PROVIDER = "databricks"  # or "openai"
AGENT_MODEL = "databricks-gpt-5"
JUDGE_MODEL = "databricks-gemini-2-5-flash"
TEMPERATURE = 1.0
EXPERIMENT_NAME = "customer-support-v2-demo"

print("✓ Configuration:")
print(f"  Provider: {PROVIDER}")
print(f"  Agent Model: {AGENT_MODEL}")
print(f"  Judge Model: {JUDGE_MODEL}")

✓ Configuration:
  Provider: databricks
  Agent Model: databricks-gpt-5
  Judge Model: databricks-gemini-2-5-flash


## Step 1: Setup MLflow Tracking

In [3]:
mlflow.openai.autolog()
using_databricks_mlflow = False
if using_databricks_mlflow:
    mlflow.set_tracking_uri("databricks")
    EXPERIMENT_NAME = f"/Users/danny.chiao@databricks.com/{EXPERIMENT_NAME}"
    mlflow.set_experiment(EXPERIMENT_NAME)
else:
    mlflow.set_tracking_uri(None)
    mlflow.set_experiment(EXPERIMENT_NAME)

2025/12/09 11:54:18 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/09 11:54:18 INFO mlflow.store.db.utils: Updating database tables
2025/12/09 11:54:18 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/09 11:54:18 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025/12/09 11:54:18 INFO alembic.runtime.migration: Running 

## Step 2: Initialize Agent (Conversation-Only)

Note: This agent handles conversations only. We'll set up evaluation separately.

In [4]:
config = AgentConfig(
    model=AGENT_MODEL,
    provider=PROVIDER,
    temperature=TEMPERATURE,
    mlflow_experiment=EXPERIMENT_NAME
)

agent = CustomerSupportAgentSimple(config)

print("✓ Agent initialized (conversation-only)")
print(f"  Provider: {config.provider}")
print(f"  Model: {config.model}")

✓ Agent initialized (conversation-only)
  Provider: databricks
  Model: databricks-gpt-5


## Step 3: Run Conversation Scenario

In [5]:
# Load printer troubleshooting scenario
scenario = get_scenario_printer_troubleshooting()

print(f"Scenario: {scenario['name']}")
print(f"Turns: {len(scenario['messages'])}")
print(f"Session ID: {scenario['session_id']}")

Scenario: Printer Troubleshooting
Turns: 4
Session ID: session-printer-001


In [6]:
# Run the conversation
# Under the hood, it tags messages with a session id (mlflow.update_current_trace(metadata={"mlflow.trace.session": session_id}))
# Then it calls into OpenAI for responses, but sends simulated pre-canned user responses
conv_result = agent.run_conversation(
    messages=scenario['messages'],
    session_id=scenario['session_id']
)

print(f"\n✓ Conversation completed")
print(f"  Turns: {conv_result['turns']}")
print(f"Now run `mlflow ui` from this directory to see the experiment in http://127.0.0.1:5000/#/experiments/1/chat-sessions/{scenario['session_id']}")


Running 4-turn conversation (Session: session-printer-001)

Turn 1/4
  User: My HP LaserJet 3000 won't turn on at all. The power light doesn't come on.
  Agent: Sorry you’re having trouble. Let’s narrow it down:

1) Power source
- Is the outlet working? Test it with another device.
- Try a different outlet (avoid power strips/surge protectors).
- Check the printer’s power cable and the removable power module (if present) are firmly seated at both ends.

2) Power components
- Inspect the cable for damage; try another compatible IEC power cable if available.
- Remove any external power switch or UPS in between.

3) Reset
- Unplug printer for 60 seconds, hold power button (if present) for 10 seconds, then reconnect and try again.

If still dead: do you hear any fans/clicks or see any brief lights? Also, confirm exact model variant (e.g., 3000n/3000dn). Likely PSU or low-voltage power supply failure; I can guide next steps.

Turn 2/4
  User: Yes, I've checked the power cable and it's plug

---

# 🎯 Evaluation Showcase: MLflow Methods

Now we'll demonstrate MLflow's evaluation capabilities step-by-step.

## Step 4: Create Session-Level Judges with `make_judge()`

This is where we showcase MLflow's judge creation API.

In [7]:
# First, let's see what the judges will evaluate
print("="*70)
print("COHERENCE JUDGE INSTRUCTIONS")
print("="*70)
print(get_coherence_judge_instructions())
print("\n" + "="*70)
print("CONTEXT RETENTION JUDGE INSTRUCTIONS")
print("="*70)
print(get_context_retention_judge_instructions())
print("\n" + "="*70)
print("\n💡 Both instructions contain {{ conversation }} - this makes them session-level!")

COHERENCE JUDGE INSTRUCTIONS
You are evaluating the coherence of a multi-turn customer support conversation.

Conversation to evaluate:
{{ conversation }}

Evaluate whether the conversation flows logically:
- Does the agent maintain context across turns?
- Are responses relevant to previous messages?
- Does the conversation follow a logical progression?
- Are there any contradictions or confusing jumps?

Provide your evaluation as:

- Value: True if the conversation is coherent and flows naturally, False if there are significant coherence issues.

- Rationale: Explain your reasoning in 2-3 sentences. Consider:
  - Context maintenance: Agent remembers what user said earlier
  - Logical flow: Each turn builds on previous turns
  - Relevance: Responses address the actual questions/issues raised
  - Consistency: No contradictions in advice or information


CONTEXT RETENTION JUDGE INSTRUCTIONS
You are evaluating context retention in a multi-turn customer support conversation.

Conversation 

In [8]:
# Configure judge model URI
if PROVIDER == "databricks":
    os.environ["OPENAI_API_KEY"] = os.environ.get("DATABRICKS_TOKEN", "")
    os.environ["OPENAI_API_BASE"] = f"{config.databricks_host}/serving-endpoints"
    judge_model_uri = f"openai:/{JUDGE_MODEL}"
else:
    judge_model_uri = JUDGE_MODEL

print(f"Judge model URI: {judge_model_uri}")

Judge model URI: openai:/databricks-gemini-2-5-flash


In [9]:
# Create judges using mlflow.genai.judges.make_judge()
print("Creating judges...\n")

coherence_judge = make_judge(
    name="conversation_coherence",
    model=judge_model_uri,
    instructions=get_coherence_judge_instructions(),
    feedback_value_type=bool
)

context_judge = make_judge(
    name="context_retention",
    model=judge_model_uri,
    instructions=get_context_retention_judge_instructions(),
    feedback_value_type=Literal["excellent", "good", "fair", "poor"]
)

print("✓ Judges created")
print(f"  Coherence judge is session-level: {coherence_judge.is_session_level_scorer}")
print(f"  Context judge is session-level: {context_judge.is_session_level_scorer}")

Creating judges...

✓ Judges created
  Coherence judge is session-level: True
  Context judge is session-level: True


## Step 5: Search for Traces with `mlflow.search_traces()`

In [10]:
# Get experiment and search for traces
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

session_traces = mlflow.search_traces(
    experiment_ids=[experiment.experiment_id],
    filter_string=f"metadata.`mlflow.trace.session` = '{scenario['session_id']}'"
)

print(f"✓ Found {len(session_traces)} traces")
print(f"  Session ID: {scenario['session_id']}")
print(f"  Each trace = 1 conversation turn")
display(session_traces[["request_time", "request", "response"]].sort_values(by="request_time", ascending=True))

✓ Found 4 traces
  Session ID: session-printer-001
  Each trace = 1 conversation turn


/var/folders/5v/7y2bspmn7fs31bnn_n8pl6v80000gp/T/ipykernel_40589/1991061166.py:4: FutureWarning: Parameter 'experiment_ids' is deprecated. Please use 'locations' instead.
  session_traces = mlflow.search_traces(


,request_time,request,response
3,1765310071036,"{'message': 'My HP LaserJet 3000 won't turn on at all. The power light doesn't come on.', 'session_id': 'session-printer-001'}","Sorry you’re having trouble. Let’s narrow it down:\n\n1) Power source\n- Is the outlet working? Test it with another device.\n- Try a different outlet (avoid power strips/surge protectors).\n- Check the printer’s power cable and the removable power module (if present) are firmly seated at both ends.\n\n2) Power components\n- Inspect the cable for damage; try another compatible IEC power cable if available.\n- Remove any external power switch or UPS in between.\n\n3) Reset\n- Unplug printer for 60 seconds, hold power button (if present) for 10 seconds, then reconnect and try again.\n\nIf still dead: do you hear any fans/clicks or see any brief lights? Also, confirm exact model variant (e.g., 3000n/3000dn). Likely PSU or low-voltage power supply failure; I can guide next steps."
2,1765310075812,"{'message': 'Yes, I've checked the power cable and it's plugged in securely to both the printer and wall outlet.', 'session_id': 'session-printer-001'}","Thanks for confirming. Let’s continue step by step:\n\n1) Outlet and cord\n- Please test that wall outlet with another device.\n- If possible, try a different IEC power cord.\n\n2) Bypass accessories\n- Plug the printer directly into the wall (no surge protector/UPS).\n\n3) Power reset\n- Unplug the printer for 60 seconds.\n- While unplugged, press/hold the power button for 10 seconds.\n- Reconnect and try powering on.\n\n4) Signs of life\n- On plug-in, do you hear any clicks/fans or see any brief LEDs?\n\nAlso, what exact model (3000, 3000n, 3000dn)? Results will tell us if the low-voltage power supply/formatter is failing."
1,1765310081083,"{'message': 'I tried a different outlet in another room and still nothing. What should I try next?', 'session_id': 'session-printer-001'}","Thanks for testing the outlet. Next steps:\n\n1) Cord and connections\n- Try a different IEC power cord (same type as PCs/monitors).\n- Reseat the power cord at the printer and wall firmly.\n\n2) Hard reset\n- Unplug printer, hold power button 15 seconds, wait 2 minutes, plug in, try again.\n\n3) Listen/look\n- When plugging in, any brief click, fan spin, or flash? Note exactly.\n\n4) Model check\n- Confirm exact model (3000/3000n/3000dn).\n\nIf still completely dead: likely low‑voltage power supply (LVPS) or DC controller failure. We can guide opening the side cover to check the fuse/LVPS assembly or recommend a repair shop/part."
0,1765310085038,"{'message': 'Okay, I'll contact HP support for a warranty replacement. Thanks for your help!', 'session_id': 'session-printer-001'}","You’re welcome! Before you call, a couple quick tips to speed things up with HP:\n\n- Have the exact model (3000/3000n/3000dn) and serial number ready.\n- Describe steps you’ve done: different outlets, bypassed surge protector, power reset, tried another IEC cord, no lights/sounds.\n- Ask about LVPS/power board failure and repair/replacement options.\n\nIf they need more diagnostics, I’m happy to guide you. Good luck!"


## Step 6: Evaluate with `mlflow.genai.evaluate()`

**Key MLflow API**: This is where the magic happens!

In [11]:
print(f"Evaluating conversation...\n")

with mlflow.start_run(run_name=scenario['name']) as run:
    eval_results = mlflow.genai.evaluate(
        data=session_traces,
        scorers=[coherence_judge, context_judge]
    )

print("✓ Evaluation complete")
result_df = eval_results.result_df
print(f"  Result DataFrame shape: {result_df.shape}")
if not using_databricks_mlflow:
    print(f"See results at http://localhost:5000/#/experiments/1/chat-sessions/session-printer-001")
    print(f"See results at http://localhost:5000/#/experiments/1/evaluation-runs?selectedRunUuid={eval_results.run_id}")

2025/12/09 11:56:01 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating conversation...



Evaluating:   0%|          | 0/4 [Elapsed: 00:00, Remaining: ?] 


✨ Evaluation completed.

Metrics and evaluation results are logged to the MLflow run:
  Run name: Printer Troubleshooting
  Run ID: 95b0d789d7834e6792751ef712d92d29

To view the detailed evaluation results with sample-wise scores,
open the Traces tab in the Run page in the MLflow UI.

✓ Evaluation complete
  Result DataFrame shape: (4, 14)
See results at http://localhost:5000/#/experiments/1/chat-sessions/session-printer-001
See results at http://localhost:5000/#/experiments/1/evaluation-runs?selectedRunUuid=95b0d789d7834e6792751ef712d92d29


In [12]:
display(result_df[["assessments"]])

,assessments
0,[]
1,"[{'assessment_id': 'a-5d95ab972b2f41008d28ba0cde071050', 'assessment_name': 'conversation_coherence', 'trace_id': 'tr-12fca87b73588c6ef88c12722119caf1', 'span_id': '3cd5f3485cabefd9', 'source': {'source_type': 'LLM_JUDGE', 'source_id': 'openai:/databricks-gemini-2-5-flash'}, 'create_time': '2025-12-09T19:56:10.598Z', 'last_update_time': '2025-12-09T19:56:10.598Z', 'feedback': {'value': True}, 'rationale': 'The conversation flows logically, with the agent consistently maintaining context and building on previous messages. The agent's responses are relevant, offering structured troubleshooting steps and adapting to the user's progress. There are no contradictions or confusing jumps, ensuring a coherent and natural progression of the support interaction.', 'metadata': {'mlflow.trace.session': 'session-printer-001', 'mlflow.assessment.sourceRunId': '95b0d789d7834e6792751ef712d92d29'}, 'valid': True}, {'assessment_id': 'a-d96c7ec67d1044edb2fa91cb8b650c7e', 'assessment_name': 'context_retention', 'trace_id': 'tr-12fca87b73588c6ef88c12722119caf1', 'span_id': '3cd5f3485cabefd9', 'source': {'source_type': 'LLM_JUDGE', 'source_id': 'openai:/databricks-gemini-2-5-flash'}, 'create_time': '2025-12-09T19:56:20.628Z', 'last_update_time': '2025-12-09T19:56:20.628Z', 'feedback': {'value': 'good'}, 'rationale': 'The agent consistently remembers the user's core problem and acknowledges their actions, like testing the outlet. While there is some minor repetition in suggesting troubleshooting steps (e.g., trying a different power cord or variations of a power reset), the conversation maintains good continuity and the agent doesn't ask for information already provided by the user. The final summary of steps for HP support demonstrates excellent recall of the entire interaction.', 'metadata': {'mlflow.trace.session': 'session-printer-001', 'mlflow.assessment.sourceRunId': '95b0d789d7834e6792751ef712d92d29'}, 'valid': True}]"
2,[]
3,[]


---

## 📊 What Just Happened?

We just demonstrated the complete MLflow evaluation workflow:

1. **`make_judge()`**: Created session-level judges with `{{ conversation }}` template
2. **`mlflow.search_traces()`**: Found all traces for the conversation
3. **`mlflow.genai.evaluate()`**: Evaluated the full conversation (not individual turns)

**Key Insight**: Session-level evaluation assesses the entire conversation holistically, enabling evaluation of:
- Conversation coherence and flow
- Context retention across turns
- Logical progression of the discussion